In [24]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pymongo import MongoClient
from pprint import pprint
import time


In [25]:
client = MongoClient('localhost',27017)
db = client['vacancies_db']
hh_vac = db.hh_vac
sj_vac = db.sj_vac

In [26]:
def hh_search(text, page, only_new = 'yes', hh_vac = hh_vac):

    user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}
    link_hh = ('https://hh.ru/search/vacancy?area=2&st=searchVacancy@enable_snippets=true&')
    
    vacs = []

    for p in range(page):
        html = requests.get(f'{link_hh}&text={text}&page={p}',  headers = user_agent).text
        parsed_html = bs(html,'lxml')
        vac_list = parsed_html.findAll('div', {'class':'vacancy-serp-item__row vacancy-serp-item__row_header'})
        df = pd.DataFrame(columns=['name', 'salary_min', 'salary_max', 'url', 'site'])

        for vac in vac_list:
            
            vac_data = {}

            main_info = vac.findChild()
            link = vac.find('a')
            if not link:
                url = 'nan'
                vac_data['name'] = ''
            else:
                vac_data['name'] = link.getText()
                url = link["href"].split('?')[0]
                vac_num = url.split('/')[3]

            compensation = vac.findParent().find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            salary = 0
            salary_min = float('nan')
            salary_max = float('nan')
            if not compensation:
                pass
            else:
                salary = compensation.getText()
                salary = re.sub(r"\s+", "", salary)
                if salary.find('от') >= 0 :
                    salary_min = int(re.findall(r'\d+',  salary)[0])

                elif salary.find('до') >= 0 :
                    salary_max = int(re.findall(r'\d+', salary)[0])

                elif salary.find('-') >= 0 :
                    salary = re.findall(r'\d+', salary)
                    salary_min = salary[0]
                    salary_max = salary[1]

              
                vac_data['salary_min'] = salary_min
                vac_data['salary_max'] = salary_max
                vac_data['url'] = url
                vac_data['vac_num'] = vac_num
                vac_data['site'] = 'HeadHunter'

                vacs.append(vac_data)
                df.append(vac_data, ignore_index = True)
                if only_new == 'yes':
                  if find_in_db('hh', vac_num) == 0:
                    hh_vac.insert_one(vac_data)
                else:
                    hh_vac.insert_one(vac_data)
            i += 1
            time.sleep(1)
    return(vacs)

In [27]:
def find_in_db(site, vac_num):
  exists = 0
  if site == 'hh':
    exists = hh_vac.count_documents({'vac_num': vac_num})
  return exists


In [28]:
def find_salary():
  salary = input('Введите минимальную зарплату: ')
  expensive_vacs = hh_vac.find({'salary_min':{'$gte':salary}}).sort('salary_min')
  print('Вакансии с зарплатой выше чем ', salary)
  for item in expensive_vacs:
    pprint(item)

In [29]:
find_salary()

Введите минимальную зарплату: 50000
Вакансии с зарплатой выше чем  50000


In [30]:
def search_vac(only_new ='yes'):

  text = input('Введите вакансию: ')
  text = text.replace(' ', '+')
  page = int(input('Введите кол-во страниц: '))
  hh_search(text, page, only_new)
  df1 = pd.DataFrame.from_dict(hh_search(text, page))
  df = pd.concat([df1], ignore_index=True)
  return df

In [33]:
print(search_vac('yes'))


Введите вакансию: Python
Введите кол-во страниц: 3
Empty DataFrame
Columns: []
Index: []
